In [1]:
import os
gpu_num = 0 # 使用 "" 来启用 CPU
os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)
# Avoid warnings from TensorFlow
tf.get_logger().setLevel('ERROR')

tf.random.set_seed(1) # Set global random seed for reproducibility
import matplotlib.pyplot as plt
import numpy as np
import pickle
import time
import mitsuba as mi
import open3d as o3d
from open3d.web_visualizer import draw

# Import Sionna RT components
from mysionna.rt import load_scene, Transmitter, Receiver, PlanarArray, Camera
from mysionna.rt.scattering_pattern import *
from sionna.utils import insert_dims

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[Open3D INFO] Resetting default logger to print to terminal.


In [2]:
scene = load_scene("Indoor/indoor.xml")

In [3]:
#################配置发端天线阵列#################
scene.tx_array = PlanarArray(num_rows=2,
                             num_cols=1,
                             vertical_spacing=0.5,
                             horizontal_spacing=0.5,
                             pattern="dipole",
                             polarization="V")

#################配置收端天线阵列#################
scene.rx_array = PlanarArray(num_rows=2,
                             num_cols=1,
                             vertical_spacing=0.5,
                             horizontal_spacing=0.5,
                             pattern="dipole",
                             polarization="V")

################创建发射机#########################
tx = Transmitter(name="tx",
                 position=[0,0,2.95])
################ 将发射机加入到场景中##############
scene.add(tx)
#################创建接收机########################
rx = Receiver(name="rx",
              position=[0,0,2.95])
################ 将接收机加入到场景中##############
scene.add(rx)
tx.look_at([0,0,0])
rx.look_at([0,0,0])

In [4]:
scene.frequency = 2.14e9 # in Hz; implicitly updates RadioMaterials
scene.synthetic_array = True # If set to False, ray tracing will be done per antenna element (slower for large arrays)

In [5]:
scene.target_names = ["door"]
scene.target_velocities = [(5.,5.,0.)]

In [6]:
p1 = LambertianPattern()
p2 = DirectivePattern(50)
scene.get("itu_plywood").scattering_coefficient = 0.4
scene.get("itu_plywood").scattering_pattern = p2
scene.get("itu_concrete").scattering_coefficient = 0.7
scene.get("itu_concrete").scattering_pattern = p1
scene.get("itu_floorboard").scattering_coefficient = 0.3
scene.get("itu_floorboard").scattering_pattern = p2
scene.get("itu_ceiling_board").scattering_coefficient = 0.7
scene.get("itu_ceiling_board").scattering_pattern = p1

In [7]:
paths = scene.compute_paths(max_depth=4,diffraction=True,scattering=True,edge_diffraction=True,scat_keep_prob=0.001)

In [8]:
# scene.preview(paths=paths)

In [9]:
v,obj=scene.compute_target_velocities(paths, return_obj_names=True)

In [10]:
subcarrier_spacing = 15e3
num_time_steps = 1
paths.apply_doppler(sampling_frequency=subcarrier_spacing, num_time_steps=num_time_steps,target_velocities=v)

In [11]:
a,tau = paths.cir()
frequency = scene.frequency
length = a.shape[5]
snr=10
phase = tf.complex(tf.zeros_like(tau),2*np.pi*frequency*tau)
e = tf.exp(phase)
e = tf.expand_dims(e, axis=-1)
a = a * e
if scene.synthetic_array:
    tau = tf.expand_dims(tau, axis=3)
    tau = tf.expand_dims(tau, axis=2)
tau_i = tf.repeat(tau,length,axis=-1)
tau_i = tf.reshape(tau_i, [tau.shape[0],tau.shape[1],tau.shape[2],tau.shape[3],tau.shape[4],length,length])
tau_j = tf.transpose(tau_i,perm=[0,1,2,3,4,6,5])
tau_i_mine_j = tau_i- tau_j
tau_i_mul_j = tau_i* tau_j
tau_i_mine_j = tf.expand_dims(tau_i_mine_j, axis=-1)
tau_i_mul_j = tf.expand_dims(tau_i_mul_j, axis=-1)
alpha = tf.expand_dims(a, axis=-2)
alpha_1 = tf.transpose(alpha,perm=[0,1,2,3,4,7,5,6])
alpha_2 = tf.transpose(alpha,perm=[0,1,2,3,4,7,6,5])
alpha_ij = tf.matmul(alpha_1,alpha_2)
alpha_ij = tf.transpose(alpha_ij,perm=[0,1,2,3,4,6,7,5])
one = tf.ones((length,length))
one = insert_dims(one, 5, 0)
one = insert_dims(one, 1, -1)
F_alpha= 2*snr*tf.math.divide_no_nan(tf.math.abs(alpha_ij),(tau_i_mul_j**2))
F_cos = (one+4*(np.pi**2)*(frequency) * tau_i_mul_j)*tf.math.cos(2*np.pi*frequency*tau_i_mine_j)
F_sin = 2*np.pi*frequency*tau_i_mine_j*tf.math.sin(2*np.pi*frequency*tau_i_mine_j)
F = F_alpha*(F_cos+F_sin)
del alpha,alpha_1,alpha_2,alpha_ij,tau_i_mine_j,tau_i_mul_j,tau_i,tau_j,F_alpha,F_cos,F_sin,one,phase,e
F = tf.transpose(F,perm=[0,1,2,3,4,7,5,6])
F = tf.reshape(F, [-1,length,length]) * 1e-18

In [12]:
crb = tf.linalg.diag_part(tf.linalg.pinv(F))
crb = tf.abs(crb)

In [13]:
objects = paths.objects
vertices = paths.vertices

num_targets = objects.shape[1]
num_sources = objects.shape[2]
num_rx = paths.a.shape[1]
num_rx_ant = paths.a.shape[2]
num_tx = paths.a.shape[3]
num_tx_ant = paths.a.shape[4]
max_num_paths = paths.a.shape[5]
max_depth = objects.shape[0]

if objects.shape[1] != num_rx*num_rx_ant:
    objects = tf.repeat(objects,int(num_rx*num_rx_ant/objects.shape[1]),axis=1)
if objects.shape[2] != num_tx*num_tx_ant:
    objects = tf.repeat(objects,int(num_tx*num_tx_ant/objects.shape[2]),axis=2)
objects = tf.reshape(objects, [max_depth,num_rx,num_rx_ant,num_tx,num_tx_ant,max_num_paths])
if vertices.shape[1] != num_rx*num_rx_ant:
    vertices = tf.repeat(vertices,int(num_rx*num_rx_ant/vertices.shape[1]),axis=1)
if vertices.shape[2] != num_tx*num_tx_ant:
    vertices = tf.repeat(vertices,int(num_tx*num_tx_ant/vertices.shape[2]),axis=2)
vertices = tf.reshape(vertices, [max_depth,num_rx,num_rx_ant,num_tx,num_tx_ant,max_num_paths,3])

In [14]:
crb = tf.where(crb==0,1,crb)
crb = tf.reshape(crb, [-1,num_rx,num_rx_ant,num_tx,num_tx_ant,num_time_steps,max_num_paths])
crb = tf.transpose(crb,perm=[0,1,2,3,4,6,5])

In [15]:
crb_b1 = tf.reduce_min(crb,axis=-1)

In [16]:
mask = paths.mask
if mask.shape[1] != num_rx*num_rx_ant:
    mask = tf.repeat(mask,int(num_rx*num_rx_ant/mask.shape[1]),axis=1)
if mask.shape[2] != num_tx*num_tx_ant:
    mask = tf.repeat(mask,int(num_tx*num_tx_ant/mask.shape[2]),axis=2)
mask = tf.reshape(mask, [-1,num_rx,num_rx_ant,num_tx,num_tx_ant,max_num_paths])
crb_b1 = tf.where(mask,crb_b1,1)
crb_b1 = tf.repeat(crb_b1,max_depth,axis=0)

indices = tf.where(objects != -1)

In [24]:
v = tf.gather_nd(vertices, indices)
c = tf.gather_nd(crb_b1, indices)
c = tf.where(c==0,1,c)
c = tf.where(c==1,0,c)
indices = tf.where(c != 0)
c = tf.gather_nd(c, indices)
v = tf.gather_nd(v, indices)
c = np.log10(c)
# normalize c
c = (c - np.min(c)) / (np.max(c) - np.min(c))
v = v.numpy()


In [34]:
# v1,idx = np.unique(v, return_index = True,axis=0)
# c1 = c[idx]

In [36]:
c_color = np.expand_dims(c, axis=-1)
c_color = np.repeat(c_color,3,axis=-1)
color_start = np.array([[75./255, 0, 130./255]])
color_start = np.repeat(color_start,c.shape[0],axis=0)
color_end = np.array([[1, 1, 0]])
color_end = np.repeat(color_end,c.shape[0],axis=0)
c_color = color_start + c_color * (color_end - color_start)

In [38]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(v)
pcd.colors = o3d.utility.Vector3dVector(c_color)
# pcd.colors = o3d.utility.Vector3dVector(np.random.rand(v.shape[0],3))
o3d.io.write_point_cloud("test.xyzrgb", pcd)

True